In [3]:
import pandas as pd

In [25]:
json_file_path = 'test.json'
df = pd.read_json(json_file_path, lines=True)

In [19]:
def determine_profit_loss(group):
    # 'main' 타입 주문 중 'open'과 'close' 상태의 주문 필터링
    open_order = group[(group['order_type'] == 'main') & (group['open_or_close'] == 'open')]
    close_order = group[(group['order_type'] == 'main') & (group['open_or_close'] == 'close')]

    # 'open'과 'close' 상태의 주문이 모두 있는지 확인
    if not open_order.empty and not close_order.empty:
        open_price = open_order['current_price'].iloc[0]
        close_price = close_order['current_price'].iloc[0]
        position = open_order['position'].iloc[0]

        if position == 'LONG':
            return 'profit' if close_price > open_price else 'loss'
        elif position == 'SHORT':
            return 'profit' if close_price < open_price else 'loss'
    if not open_order.empty and close_order.empty:
        return 'activate'
    
    if open_order.empty and not close_order.empty:
        return "open_empty"
    
    return 'unknown'  # 'open' 또는 'close' 주문이 누락된 경우


In [29]:
# split 주문 분석
def split_order_analysis(group, split_status, split_type):
    return len(group[(group['order_type'] == 'split') & (group['open_or_close'] == split_status) & (group['split_type'] == split_type)])
# orderId를 입력으로 받아 해당 orderId만 필터링하여 반환하는 함수

def filter_df(_df,key,filter_value):
    filtered_df = _df[_df[key] == filter_value]
    return filtered_df

In [46]:
filter_df(df,"order_id",12170825073)

,timestamp,order_type,open_or_close,order_id,user_id,symbol,position,side,quantity,average_price,trigger_price,current_price,profit_price,split_type,split_step,close_type,pnl_trigger_price,water_trigger_price,bull_trigger_price
20,2024-03-25 19:25:41,main,close,12170825073,hamoni2,BANDUSDT,LONG,BUY,14.8,0.0,2.18736,2.1875,NaN,NaN,NaN,profit,None,None,None


In [53]:
def aggregate_results(group):
    return pd.Series({
        'symbol': group['symbol'].iloc[0],  # 첫 번째 symbol 값
        'position': group['position'].iloc[0],  # 첫 번째 position 값
        'profit_loss': determine_profit_loss(group),
        'split_open_water': split_order_analysis(group, 'open', 'water'),
        'split_close_water': split_order_analysis(group, 'close', 'water'),
        'split_open_bull': split_order_analysis(group, 'open', 'bull'),
        'split_close_bull': split_order_analysis(group, 'close', 'bull'),
    })

# apply 함수를 사용할 때 include_groups=False를 추가하거나 경고를 무시
results = df.groupby('order_id').apply(aggregate_results, include_groups=False).reset_index()


In [54]:
results[(results["profit_loss"] == "loss") | (results["profit_loss"] == "profit")]

,order_id,symbol,position,profit_loss,split_open_water,split_close_water,split_open_bull,split_close_bull
2,875895396,TRUUSDT,SHORT,loss,35,25,0,0
3,877317405,TRUUSDT,LONG,profit,11,11,5,5
4,900472957,TUSDT,LONG,profit,0,0,5,5
9,3333774111,JASMYUSDT,SHORT,loss,20,10,0,0
10,3334553266,JASMYUSDT,LONG,profit,1,1,5,5
13,8505524823,APTUSDT,SHORT,loss,21,11,1,0
14,8505924009,APTUSDT,LONG,profit,6,6,5,5
22,633327755710356224,RSRUSDT,LONG,profit,2,2,5,5


In [42]:
filter_df(results,"profit_loss","unknown")

,order_id,symbol,position,profit_loss,split_open_water,split_close_water,split_open_bull,split_close_bull
0,672477120,IDEXUSDT,SHORT,unknown,5,4,0,0
1,833333742,KEYUSDT,SHORT,unknown,7,6,0,0
5,1029665539,CKBUSDT,SHORT,unknown,2,0,0,0
6,2703421782,CTKUSDT,SHORT,unknown,0,0,1,2
7,3293571738,ARPAUSDT,SHORT,unknown,23,14,0,0
8,3294776039,ARPAUSDT,LONG,unknown,0,0,3,1
11,3527806367,STXUSDT,LONG,unknown,18,12,0,0
12,6833619104,MKRUSDT,LONG,unknown,0,0,0,0
15,10898466071,XMRUSDT,SHORT,unknown,1,0,0,0
16,11592237640,TRXUSDT,LONG,unknown,1,1,1,1


         total_orders  profits  losses  water_splits  bull_splits  \
user_id                                                             
hamoni2            20      107     175           217           43   

         closed_splits  
user_id                 
hamoni2            114  


C:\Users\seeti\AppData\Local\Temp\ipykernel_1416\351137846.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_stats = df.groupby('user_id').apply(lambda x: pd.Series({
